In [4]:
%pip install tensorflow-metal

  Using cached tensorflow_metal-1.1.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (1.2 kB)
Using cached tensorflow_metal-1.1.0-cp311-cp311-macosx_12_0_arm64.whl (1.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:

"""
This script has the method
preprocess_data(X, Y):
"""
import tensorflow.keras as K

#import keras as K


def preprocess_data(X, Y):
        """ This method has the preprocess to train a model """
        X = X.astype('float32')
        X_p = K.applications.vgg16.preprocess_input(X)
        Y_p = K.utils.to_categorical(Y, 10)
        return(X_p, Y_p)

if __name__ == "__main__":
    (Xt, Yt), (X, Y) = K.datasets.cifar10.load_data()
    X_p, Y_p = preprocess_data(Xt, Yt)
    Xv_p, Yv_p = preprocess_data(X, Y)
    base_model = K.applications.vgg16.VGG16(include_top=False,
                                            weights='imagenet',
                                            pooling='avg'
                                            )

    model= K.Sequential()
    model.add(K.layers.UpSampling2D())
    model.add(base_model)
    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(512, activation=('relu'))) 
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.Dense(256, activation=('relu')))
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.Dense(10, activation=('softmax')))
    callback = []
    
    def decay(epoch):
        """ This method create the alpha"""
        return 0.001 / (1 + 1 * 30)
    callback += [K.callbacks.LearningRateScheduler(decay, verbose=1)]
    callback += [K.callbacks.ModelCheckpoint('cifar10.h5',
                                             save_best_only=True,
                                             mode='min'
                                             )]
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x=X_p, y=Y_p,
              batch_size=32,
              validation_data=(Xv_p, Yv_p),
              epochs=10, shuffle=True,
              callbacks=callback,
              verbose=1
              )

2024-03-20 21:15:25.654111: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-03-20 21:15:25.654138: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-20 21:15:25.654142: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-20 21:15:25.654178: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-20 21:15:25.654196: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)



Epoch 1: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 1/10


2024-03-20 21:15:26.540484: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-03-20 21:15:27.087398: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - ETA: 0s - loss: 2.0657 - accuracy: 0.5969

2024-03-20 21:17:41.658619: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 143s 91ms/step - loss: 2.0657 - accuracy: 0.5969 - val_loss: 1.0813 - val_accuracy: 0.7998 - lr: 3.2258e-05

Epoch 2: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 2/10
1563/1563 [==============================] - 141s 90ms/step - loss: 1.9045 - accuracy: 0.7739 - val_loss: 3.8308 - val_accuracy: 0.7330 - lr: 3.2258e-05

Epoch 3: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 3/10
1563/1563 [==============================] - 141s 90ms/step - loss: 1.5923 - accuracy: 0.8184 - val_loss: 1.6739 - val_accuracy: 0.8185 - lr: 3.2258e-05

Epoch 4: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 4/10
1563/1563 [==============================] - 141s 90ms/step - loss: 1.1416 - accuracy: 0.8577 - val_loss: 0.9429 - val_accuracy: 0.8623 - lr: 3.2258e-05

Epoch 5: LearningRateScheduler setting learning rate to 3.2258064516129034e-05.
Epoch 5/10
1563/1563 [=========

In [3]:
from keras.models import load_model

model = load_model('cifar10.h5')

In [4]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

predictions = model.predict(Xv_p)
predicted_labels = np.argmax(predictions, axis=1)

# Calculating metrics
print("Confusion Matrix:")
conf_matrix = confusion_matrix(np.argmax(Yv_p, axis=1), predicted_labels)
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(np.argmax(Yv_p, axis=1), predicted_labels))


  5/313 [..............................] - ETA: 5s 

2024-03-20 21:40:15.690843: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 7s 21ms/step
Confusion Matrix:
[[926  11  16   3   2   2   1   7  26   6]
 [  3 978   0   0   1   0   0   0   6  12]
 [ 18   0 906  12  20  21   6  15   1   1]
 [  7   1  31 731  23 158  15  24   8   2]
 [  4   2  35  12 891  11   9  33   3   0]
 [  2   0  15  42  12 898   5  24   1   1]
 [  6   1  29  21   8  18 910   3   2   2]
 [  5   1   7   2  15  16   0 953   1   0]
 [ 23   7   6   0   0   1   0   3 958   2]
 [ 13  81   0   4   1   1   0   2  17 881]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      1000
           1       0.90      0.98      0.94      1000
           2       0.87      0.91      0.89      1000
           3       0.88      0.73      0.80      1000
           4       0.92      0.89      0.90      1000
           5       0.80      0.90      0.84      1000
           6       0.96      0.91      0.94      1000
           7       0.90      0.95     